In [8]:
#@title Clickbait-detector

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML


# Load the trained model and tokenizer
model_path = '/content/drive/MyDrive/clickbait/deberta-baseTrainingCheckpoints/CheckPointModel_deberta_clickbait0/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Function to classify clickbait
def classify_clickbait(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).squeeze().tolist()
    clickbait_prob = probabilities[1]
    return clickbait_prob

# Create a larger text area widget for the headline input
text_input = widgets.Textarea(
    value='',
    placeholder='Enter headline here',
    description='Headline:',
    disabled=False,
    layout=widgets.Layout(width='45%', height='60px')  # Set the width and height of the text area
)

# Create a button widget for submitting the text
submit_button = widgets.Button(description="Submit", button_style='success',
                               layout=widgets.Layout(width='13%'))

# Create a reset button to clear the graph and result
reset_button = widgets.Button(description="Reset", button_style='warning',
                              layout=widgets.Layout(width='13%'))

# Create output widgets for displaying the result and the graph
output_result = widgets.Output()
output_graph = widgets.Output()

# Display the widgets
display(text_input)
display(widgets.HBox([submit_button, reset_button]))
display(output_result)
display(output_graph)

def on_submit(b):
    test_text = text_input.value
    if test_text:
        probabilities = classify_clickbait(test_text)
        # Present the result with style
        if probabilities < 0.5:
            result_text = "This headline is Clickbait"
            # icon = render_icon('exclamation-circle', color='red')
        else:
            result_text = "This headline is Non-Clickbait"
            # icon = render_icon('check-circle', color='green')

        with output_result:
          display(HTML(f'<div style="font-size:20px; font-weight:bold;">{result_text}</div>'))

        # Plot the probability distribution

        values = [1 - probabilities, probabilities]
        labels = ['Clickbait (Prob. ' + str(round(values[0], 2)) + ')',
                  'Non-Clickbait (Prob. ' + str(round(values[1], 2)) + ')']
        with output_graph:
            plt.figure(figsize=(7, 5))
            plt.bar(labels, values, color=['red', 'green'])
            plt.ylabel('Probability')
            plt.title('Clickbait Detection')
            plt.ylim(0, 1)
            plt.show()


# Define a function to clear the result and the graph
def clear_output(b):
    output_result.clear_output()
    output_graph.clear_output()
    text_input.value = ''  # Clear the content of the text area

# Assign the on_submit function to the submit button
submit_button.on_click(on_submit)

# Assign the clear_output function to the reset button
reset_button.on_click(clear_output)

Textarea(value='', description='Headline:', layout=Layout(height='60px', width='45%'), placeholder='Enter head…

Output()

Output()